<figure>
  <IMG SRC="Logo.png"  WIDTH=150 ALIGN="right">
</figure>

## Projections of mean sea level and tides<br>
### Six Dutch stations
<b>Prepared by: H.G. Voortman</b>


<figure>
    <IMG SRC="https://www.python.org/static/community_logos/python-powered-w-200x80.png"  WIDTH=100 ALIGN="right">
</figure>

#### Description

#### References
- Provided in report

#### Packages

In [1]:
# General packages
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as sq
import numpy as np
#import scipy.signal as signal
#import datetime as dt
import hvec_stat.general_fit as gf
#import hvec_stat.gof as gof
import hvec_stat.support as sup
import models as mdl
import sigfig as sf

In [2]:
# Settings
plt.rcParams['axes.grid'] = True
figsize = (20, 18)

#### Connect databases and import data

##### Processed data Rijkswaterstaat

In [3]:
# Connect database
conn_str = os.getenv('DATAPATH') + 'RWS_processed.db'
cnxn = sq.connect(conn_str, detect_types = True)

In [4]:
pd.read_sql('SELECT * FROM sqlite_master', cnxn)

,type,name,tbl_name,rootpage,sql
0,table,const_yr,const_yr,2,"CREATE TABLE ""const_yr"" (\n""naam"" TEXT,\n ""le..."
1,index,ix_const_yr_naam_level_1,const_yr,3,"CREATE INDEX ""ix_const_yr_naam_level_1""ON ""con..."


In [5]:
# Read table with observed water levels; complete years only
sql = (
    "SELECT * "
    "FROM 'const_yr' "
    "WHERE naam IN ('Delfzijl', 'Harlingen', "
    "'Den Helder', 'IJmuiden',  "
    "'Hoek van Holland', 'Vlissingen') "
    "AND (count>650) "
#    "AND (set == 'Ftested3') "
)
df = pd.read_sql(sql, cnxn) #.groupby('YEAR').mean()

In [6]:
cnxn.close()

In [7]:
df.columns = df.columns.str.replace('_ampl', '')
df.columns

Index(['naam', 'level_1', 'z0', 'zmean', 'count', 'M2', 'S2', 'Rsq_adj',
       'MHWS', 'MLWS', 'MHWN', 'MLWN', 'year', 'year_start', 'set', 'M4', 'O1',
       'K2', 'K1', 'P1', 'N2', 'M6', 'MU2', 'L2', '2MS6', 'MS4', 'NU2', 'SA'],
      dtype='object')

In [8]:
#df = df[np.abs(df['z0']) < 1e2]
#df = df[np.abs(df['M2']) < 1e2]
tsplit = 2021
df = df[df['year'].between(1945, tsplit)]

In [9]:
names = [
    'Delfzijl',
    'Harlingen',
    'Den Helder',
    'IJmuiden',
    'Hoek van Holland',
    'Vlissingen'
]

In [10]:
df = df[df['set'] == 'Ftested3']

#### Calibrate per model

In [11]:
param = pd.DataFrame(columns = [
    'name',
    'var',
    'model',
    'intercept',
    'slope',
    'acceleration',
    'A_885',
    'A_1861',
    'Rsqadj',
    'p-value'], dtype = float)

In [12]:
vars = ['z0', 'MHWS', 'MLWS']

In [13]:
tmp = pd.DataFrame(columns = ['name'])

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
Rsqadj = []
pvalue = []

for vr in vars:
    for nm in names:
        """
        if nm == 'Delfzijl':
            if vr != 'z0':
                continue
        """
        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model1,
            data['year'],
            data[vr]
        )

        p = res[0]
        
        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('Model 1')
        intercept.append(p[0])
        slope.append(p[1])
        Rsqadj.append(res[5])

        # F-test
        pvalue.append(
            sup.Ftest_classic_direct(
                mdl.model1, 
                data['year'], data[vr],
                method = 'Bence', alpha = 0.05)['p'])        


tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
tmp['Rsqadj'] = Rsqadj
tmp['p-value'] = pvalue

param = pd.concat([param, tmp])

In [14]:
param

,name,var,model,intercept,slope,acceleration,A_885,A_1861,Rsqadj,p-value
0,Delfzijl,z0,Model 1,0.044153,0.002349,NaN,NaN,NaN,0.696865,1.898048e-12
1,Harlingen,z0,Model 1,0.002755,0.001681,NaN,NaN,NaN,0.515447,8.181620e-09
2,Den Helder,z0,Model 1,-0.050433,0.001633,NaN,NaN,NaN,0.541191,1.652660e-07
3,IJmuiden,z0,Model 1,-0.016989,0.001610,NaN,NaN,NaN,0.535129,2.208119e-06
4,Hoek van Holland,z0,Model 1,0.021268,0.002252,NaN,NaN,NaN,0.706257,1.293392e-10
5,Vlissingen,z0,Model 1,-0.038328,0.001590,NaN,NaN,NaN,0.636268,1.553706e-08
6,Delfzijl,MHWS,Model 1,1.667551,0.004960,NaN,NaN,NaN,0.841500,4.423367e-11
7,Harlingen,MHWS,Model 1,1.021635,0.002514,NaN,NaN,NaN,0.678642,8.711402e-15
8,Den Helder,MHWS,Model 1,0.762417,0.002189,NaN,NaN,NaN,0.618041,1.981216e-08
9,IJmuiden,MHWS,Model 1,0.816513,0.001886,NaN,NaN,NaN,0.528380,3.969082e-07


In [15]:
tmp = pd.DataFrame(columns = ['name'])

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
acceleration = []
Rsqadj = []
pvalue = []

for vr in vars:
    for nm in names:
        """if nm == 'Delfzijl':
            if vr != 'z0':
                continue
        """

        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model2,
            data['year'],
            data[vr]
        )

        p = res[0]

        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('Model 2')
        intercept.append(p[0])
        slope.append(p[1])
        acceleration.append(p[2])
        Rsqadj.append(res[5])

        # F-test
        pvalue.append(
            sup.Ftest_reduced_direct(
                mdl.model2, mdl.model1, 
                data['year'], data[vr],
                method = 'Bence', alpha = 0.05)['p'])        

tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
tmp['acceleration'] = acceleration
tmp['Rsqadj'] = Rsqadj
tmp['p-value'] = pvalue

param = pd.concat([param, tmp])

In [16]:
tmp = pd.DataFrame(columns = ['name'])

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
acceleration = []
A_885 = []
A_1861 = []
Rsqadj = []
pvalue = []

for vr in vars:
    for nm in names:
        """
        if nm == 'Delfzijl':
            if vr != 'z0':
                continue
        """
        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model3,
            data['year'],
            data[vr]
        )

        p = res[0]

        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('Model 3')
        intercept.append(p[0])
        slope.append(p[1])
        #acceleration.append(p[2])
        A_885.append(np.sqrt(p[2] ** 2 + p[3] ** 2))
        A_1861.append(np.sqrt(p[4] ** 2 + p[5] ** 2))
        Rsqadj.append(res[5])

        # F-test
        pvalue.append(
            sup.Ftest_reduced_direct(
                mdl.model3, mdl.model1, 
                data['year'], data[vr],
                method = 'Bence', alpha = 0.05)['p'])
        

tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
#tmp['acceleration'] = acceleration
tmp['A_885'] = A_885
tmp['A_1861'] = A_1861
tmp['Rsqadj'] = Rsqadj
tmp['p-value'] = pvalue

param = pd.concat([param, tmp])

tmp = pd.DataFrame()

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
acceleration = []
A_885 = []
A_1861 = []
Rsqadj = []
pvalue = []

for vr in vars:
    for nm in names:
        """
        if nm == 'Delfzijl':
            if vr != 'z0':
                continue
        """
        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model4,
            data['year'],
            data[vr]
        )

        p = res[0]

        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('Model 4')
        intercept.append(p[0])
        slope.append(p[1])
        acceleration.append(p[2])
        A_885.append(np.sqrt(p[2] ** 2 + p[3] ** 2))
        A_1861.append(np.sqrt(p[4] ** 2 + p[5] ** 2))
        Rsqadj.append(res[5])

        # F-test
        pvalue.append(
            sup.Ftest_reduced_direct(
                mdl.model4, mdl.model1, 
                data['year'], data[vr],
                method = 'Bence', alpha = 0.05)['p'])
        

tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
tmp['acceleration'] = acceleration
tmp['A_885'] = A_885
tmp['A_1861'] = A_1861
tmp['Rsqadj'] = Rsqadj
tmp['p-value'] = pvalue

param = pd.concat([param, tmp])

In [18]:
param.loc[param['var'] == 'z0', ['name', 'model', 'slope', 'acceleration', 'A_885', 'A_1861']].to_excel(r'../Data/empirical.xlsx')

In [19]:
# Set units (length mm and time centuries) for publication table
param['intercept'] = (1000 * param['intercept']).round()
param['slope'] = (1e5 * param['slope']).round()
param['acceleration'] = (1e7 * param['acceleration']).round()
param['A_885'] = (1e3 * param['A_885']).round()
param['A_1861'] = (1e3 * param['A_1861']).round()
param['Rsqadj'] = (1e2 * param['Rsqadj']).round()
param['p-value'] = (1e2 * param['p-value']).apply(lambda x: sf.round(x, 2))

In [20]:
def func(x):
    if x < 1:
        return '< 1'
    if x < 5:
        return '> 1 and < 5'
    return round(x, 0)

In [21]:
param['p-value'] = param['p-value'].apply(lambda x: func(x))

In [22]:
param.sort_values(by = ['name', 'model'], inplace = True)

In [23]:
param

,name,var,model,intercept,slope,acceleration,A_885,A_1861,Rsqadj,p-value
0,Delfzijl,z0,Model 1,44.0,235.0,NaN,NaN,NaN,70.0,< 1
6,Delfzijl,MHWS,Model 1,1668.0,496.0,NaN,NaN,NaN,84.0,< 1
12,Delfzijl,MLWS,Model 1,-1579.0,-26.0,NaN,NaN,NaN,-10.0,89.0
0,Delfzijl,z0,Model 2,35.0,235.0,189.0,NaN,NaN,71.0,8.0
6,Delfzijl,MHWS,Model 2,1681.0,496.0,-282.0,NaN,NaN,85.0,15.0
...,...,...,...,...,...,...,...,...,...,...
11,Vlissingen,MHWS,Model 3,2181.0,205.0,NaN,7.0,16.0,65.0,31.0
17,Vlissingen,MLWS,Model 3,-2258.0,120.0,NaN,3.0,32.0,48.0,< 1
5,Vlissingen,z0,Model 4,-43.0,162.0,87.0,5.0,13.0,69.0,6.0
11,Vlissingen,MHWS,Model 4,2190.0,205.0,-192.0,6.0,11.0,67.0,15.0


In [24]:
param[param['var'] == 'z0'].drop(['var', 'intercept'], axis = 1).to_excel(r'../results/fitted_models_z0.xlsx', index = False)
param[param['var'] == 'MHWS'].drop(['var', 'intercept'], axis = 1).to_excel(r'../results/fitted_models_MHWS.xlsx', index = False)
param[param['var'] == 'MLWS'].drop(['var', 'intercept'], axis = 1).to_excel(r'../results/fitted_models_MLWS.xlsx', index = False)


#### End script
Prepared by HVEC lab, 2022